# Challenge - SD207

## Groupe : WEI Chen ,  LUO Xi

# Read data

In [1]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.qda import QDA
from sklearn.decomposition import PCA


D:\Anaconda\lib\site-packages\sklearn\qda.py:6: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [3]:
df1 = np.loadtxt("train.txt", dtype = "string")
tr_fn = df1[:,0]
tr_labels_name = df1[:,1]
u,tr_labels = np.unique(tr_labels_name, return_inverse= True)

In [18]:
for i in range(tr_fn.shape[0]):
    y,sr = librosa.load(tr_fn[i])
    mfcc =librosa.feature.mfcc(y = y, n_fft = 8192,hop_length = 8192, n_mfcc = 20 )
    if i == 0:
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        X_tr = np.c_[mfcc.T, mfcc_delta.T, mfcc_delta2.T, cqt.T]
        y_tr = tr_labels[i] * np.array([1]* mfcc.shape[1])
    else: 
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        tmp = np.c_[mfcc.T, mfcc_delta.T,mfcc_delta2.T, cqt.T]
        X_tr  = np.concatenate((X_tr,tmp), axis = 0)
        y_tr = np.concatenate((y_tr, tr_labels[i] * np.array([1]* mfcc.shape[1])), axis = 0)

In [19]:
print X_tr.shape
print y_tr.shape

(47142L, 324L)
(47142L,)


In [20]:
df2 = np.loadtxt("dev.txt", dtype = "string")
tt_fn = df2[:,0]
tt_lb = df2[:,1]

In [21]:
u,tt_labels= np.unique(tt_lb, return_inverse= True)
print tt_labels.shape

(290L,)


In [23]:
for i in range(tt_fn.shape[0]):
    y,sr = librosa.load(tt_fn[i])
    mfcc = librosa.feature.mfcc(y = y, n_fft = 8192,hop_length = 8192, n_mfcc = 20 )
    if i == 0:
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        X_tt = np.c_[mfcc.T, mfcc_delta.T,mfcc_delta2.T, cqt.T]
        y_tt = tt_labels[i] * np.array([1]* mfcc.shape[1])
    else: 
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        tmp = np.c_[mfcc.T, mfcc_delta.T,mfcc_delta2.T, cqt.T] 
        X_tt  = np.concatenate((X_tt,tmp), axis = 0)
        y_tt = np.concatenate((y_tt, tr_labels[i] * np.array([1]* mfcc.shape[1])), axis = 0)

In [25]:
print X_tt.shape
print y_tt.shape

(23490L, 324L)
(23490L,)


In [24]:
df3 = np.loadtxt("test_files.txt", dtype = "string")
for i in range(df3.shape[0]):
    y,sr = librosa.load(df3[i])
    mfcc = librosa.feature.mfcc(y = y, n_fft = 8192,hop_length = 8192, n_mfcc = 20 )
    if i == 0:
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        X_v = np.c_[mfcc.T, mfcc_delta.T,mfcc_delta2.T,cqt.T]
    else: 
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
        cqt = librosa.cqt(y = y, sr = sr, hop_length= 8192, fmin = 5, n_bins = 24*11, bins_per_octave= 24, real = True)
        tmp = np.c_[mfcc.T, mfcc_delta.T,mfcc_delta2.T,cqt.T]
        X_v  = np.concatenate((X_v,tmp), axis = 0)

# Majority Vote Weighted

In [26]:
def majority_vote(y_pred, y_true, w, pas):
    ac = 0
    scr = []
    for i in range(y_true.shape[0]):
        counts = np.bincount(y_pred[i*pas:(i+1)*pas],weights = w[i*pas:(i+1)*pas])
        scr.append(np.max(counts)/pas)
        if np.argmax(counts) == y_true[i]:
            ac = ac + 1
            
    return [ac/float(y_true.shape[0]),scr] 

# Store Data

In [33]:
def Store(y_pred,w,pas):
    y_st = np.zeros((298,))
    scr = []
    for i in range(298):
        counts = np.bincount(y_pred[i*pas:(i+1)*pas], weights=w[i*pas:(i+1)*pas])
        y_st[i] = np.argmax(counts)
        scr.append(np.max(counts)/np.sum(counts))
    np.savetxt('y_pred.txt', y_st, fmt='%d')
    return [y_st,scr]

In [13]:
 X_train, X_test, y_train, y_test = train_test_split(np.concatenate((X_tr,X_tt),axis = 0), 
                                                     np.concatenate((y_tr,y_tt),axis  = 0), test_size=0.9, random_state=42)

In [27]:
X_train = np.concatenate((X_tr,X_tt), axis = 0)
y_train = np.concatenate((y_tr,y_tt), axis = 0)
print X_train.shape
print y_train.shape

(70632L, 324L)
(70632L,)


# Training... (PCA, MLP, bagging QDA...)

In [60]:
pca = PCA(n_components=300)
X_pca = pca.fit_transform(X_train)

#clf = MLPClassifier(hidden_layer_sizes= (200,), random_state = 1, warm_start= True )
#clf = BaggingClassifier(mlp, n_estimators= 30, max_samples=0.7, max_features= 0.7)
#clf1.fit(X_train,y_train)
#clf2 = MLPClassifier(hidden_layer_sizes=(190,))
#clf.fit(X_pca,y_train) 
#print clf.score(X_test,y_test)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=True)

In [63]:
mlp = MLPClassifier(hidden_layer_sizes= (180,), random_state = 1 )
clf = BaggingClassifier(mlp, n_estimators= 20, max_samples=0.7, max_features= 0.5, n_jobs = True)
clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(180,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.7, n_estimators=20, n_jobs=True, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [29]:
clf = MLPClassifier(hidden_layer_sizes= (200,), random_state = 1, warm_start= True )
 
print cross_val_score(clf,X_train, y_train, cv = 3, n_jobs = 3).mean()

0.514313625552


In [64]:
y_pred2 = clf.predict(X_tt)
proba = np.amax(clf.predict_proba(X_tt), axis = 1)
print majority_vote(y_pred2,tt_labels, proba,81)[0]

0.924137931034


In [65]:

y_pred3 = clf.predict(X_v)
proba3 = np.amax(clf.predict_proba(X_v), axis = 1)
print Store(y_pred3,proba3,81)[0]

[ 10.   8.   3.   7.   1.   5.   7.  14.   3.   7.  10.  12.   6.  14.  12.
   5.  12.   8.   0.   8.   8.   6.   7.  12.  12.   8.   6.   8.  11.   3.
  14.   1.  10.  12.   8.   5.   3.  14.  14.   4.   7.   0.   0.   7.   6.
  11.  12.   6.   3.   8.  11.   6.  12.  12.   6.  10.   8.   6.   5.  11.
   7.  12.  14.  11.   6.  14.   9.   7.   3.   4.   0.  14.  11.   7.  14.
   6.   1.   9.   1.  10.  12.   0.   4.   6.  13.   3.  14.  13.   1.   9.
  12.  13.  10.   6.  14.   1.  12.  10.   5.  14.   1.  13.   6.   6.   7.
  14.   7.   7.   5.  14.   4.  12.  14.  13.   3.   0.   0.   9.   9.  10.
  13.   3.   4.   6.   1.   1.   1.  12.   4.   5.  14.  10.   1.  10.   6.
   6.  11.  12.  12.  14.  12.   5.  12.   8.   3.   3.   9.   7.   1.   8.
   7.   3.   3.  13.   9.  13.  12.  12.   1.   6.   7.   4.   9.   9.  14.
   1.   9.   5.  10.   7.   1.  13.   0.   0.  11.  14.   8.   7.  12.  12.
   5.   6.   4.  13.   5.   6.   4.  14.   7.   9.  14.   7.   4.  11.   9.
   3.   9.  